In [37]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [38]:
EN_THRESHOLD = 1.0
SI_THRESHOLD = 1.0

WITHOUT_SI_MODIFIERS = True

CLASSIFY_COL_NAME = 'script'+str(EN_THRESHOLD)+'-'+str(SI_THRESHOLD)

In [39]:
import pandas as pd
url = '/content/drive/Shareddrives/Lingua/Datasets/Sinhala-English Code Mixed Code Switched/sentence -script -latin-sinhala-mixed - sentence.csv'
tags_ind = ['Sinhala', 'English', 'Sin-Eng', 'Eng-Sin', 'Mixed', 'NameEntity', 'Symbol']

irrelevant = [5,6]
latin = [1,2]
sinhala = [0,3]
both = [4]

latinalphabet = 'abcdefghijklmnopqrstuvwxyzABCDEFGHIJKLMNOPQRSTUVWXYZ'

df = pd.read_csv(url)
tags = []
tokens = []
for labels in df['tags']:
  tags.append(eval(labels))
for toks in df['tokens']:
  tokens.append(eval(toks))

df_start = pd.DataFrame({'tokens':tokens,'tags':tags})
row_count = df.shape[0]
df_start

,tokens,tags
0,"[▁Amma, ge, ▁Ada, ray, ta, ❤, ️, E, ka, ▁Da, w...","[2, 2, 2, 2, 2, 6, 6, 2, 2, 2, 2, 2, 2, 2, 2, ..."
1,"[We, need, IPL, Champions, leak, data, offers,...","[1, 1, 5, 1, 1, 1, 1, 6, 1, 1, 1, 6, 1, 1, 1, ..."
2,"[#VPN, #ummmaaa, #proud_be]","[1, 2, 1]"
3,"[chandimal, .., uuu, thama, mulu, tem, ekama, ...","[5, 6, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2]"
4,[sltgo],[5]
...,...,...
13513,"[Me, mase, web, family, plus, packge, eke, tot...","[2, 2, 1, 1, 1, 1, 2, 1, 2, 5, 2, 2]"
13514,"[munge, Dan, data, kapana, widiye, awulk, thiy...","[2, 2, 1, 2, 2, 2, 2]"
13515,"[I, didn't, received, loyalty, data, past, mon...","[1, 1, 1, 1, 1, 1, 1, 6, 1, 1, 1, 1, 1, 1, 1, ..."
13516,"[worst, service, ever, !, !, !, stolen, more, ...","[1, 1, 1, 6, 6, 6, 1, 1, 1, 5, 1, 1, 1, 1]"


In [40]:
df[CLASSIFY_COL_NAME] = ['' for i in range(row_count)]


In [41]:
df_classified = df_start.copy()
df_classified['class'] = ['' for i in range(row_count)]
df_classified

,tokens,tags,class
0,"[▁Amma, ge, ▁Ada, ray, ta, ❤, ️, E, ka, ▁Da, w...","[2, 2, 2, 2, 2, 6, 6, 2, 2, 2, 2, 2, 2, 2, 2, ...",
1,"[We, need, IPL, Champions, leak, data, offers,...","[1, 1, 5, 1, 1, 1, 1, 6, 1, 1, 1, 6, 1, 1, 1, ...",
2,"[#VPN, #ummmaaa, #proud_be]","[1, 2, 1]",
3,"[chandimal, .., uuu, thama, mulu, tem, ekama, ...","[5, 6, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2]",
4,[sltgo],[5],
...,...,...,...
13513,"[Me, mase, web, family, plus, packge, eke, tot...","[2, 2, 1, 1, 1, 1, 2, 1, 2, 5, 2, 2]",
13514,"[munge, Dan, data, kapana, widiye, awulk, thiy...","[2, 2, 1, 2, 2, 2, 2]",
13515,"[I, didn't, received, loyalty, data, past, mon...","[1, 1, 1, 1, 1, 1, 1, 6, 1, 1, 1, 1, 1, 1, 1, ...",
13516,"[worst, service, ever, !, !, !, stolen, more, ...","[1, 1, 1, 6, 6, 6, 1, 1, 1, 5, 1, 1, 1, 1]",


In [42]:
def check_only_numbers(string):
    return string.isdigit()

In [43]:
import re

def remove_symbols_and_numbers(string):
    # pattern = r'[^a-zA-Z\s]'
    pattern = r"[-()\"#/@;:<>{}`+=~|_▁.!?,1234567890]"
    clean_string = re.sub(pattern, '', string)
    return clean_string

In [44]:
def is_sinhala(char):
  # https://unicode.org/charts/PDF/U0D80.pdf
  return ord(char) >= 0x0D80 and ord(char) <= 0x0DFF

In [45]:
def get_chars(word, without_si_modifiers = True):
  mods = [0x0DCA,0x0DCF,0x0DD0,0x0DD1,0x0DD2,0x0DD3,0x0DD4,0x0DD5,0x0DD6,0x0DD7,0x0DD8,0x0DD9,0x0DDA,0x0DDB,0x0DDC,0x0DDD,0x0DDE,0x0DDF,0x0DF2,0x0DF3]
  if without_si_modifiers:
    return [char for char in list(word) if ord(char) not in mods]
  else:
    return list(word)

In [46]:

classes = []
for index, row in df_start.iterrows():
  tokens = row['tokens']
  tags = row['tags']
  total_chars = 0
  latin_char_count = 0
  sin_char_count = 0
  for t_i,t in enumerate(tokens):
    if tags[t_i] in [6]:
      continue
    if tags[t_i] == 5 and check_only_numbers(t):
      continue
    token_list = get_chars(remove_symbols_and_numbers(t),without_si_modifiers = WITHOUT_SI_MODIFIERS)
    token_len = len(token_list)
    total_chars += token_len
    for ch in token_list:
      if is_sinhala(ch):
        sin_char_count += 1
      else:
        latin_char_count += 1
  if total_chars == 0:
     classes.append('Symbol')
  else:
    en_percentage = latin_char_count/total_chars
    si_percentage = sin_char_count/total_chars
    if en_percentage >= EN_THRESHOLD:
      classes.append('Latin')
    elif si_percentage >= SI_THRESHOLD:
      classes.append('Sinhala')
    elif en_percentage < EN_THRESHOLD and si_percentage < SI_THRESHOLD:
      classes.append('Mixed')

In [47]:
df_classified['class'] = classes
df[CLASSIFY_COL_NAME] = classes

In [48]:
df_classified

,tokens,tags,class
0,"[▁Amma, ge, ▁Ada, ray, ta, ❤, ️, E, ka, ▁Da, w...","[2, 2, 2, 2, 2, 6, 6, 2, 2, 2, 2, 2, 2, 2, 2, ...",Latin
1,"[We, need, IPL, Champions, leak, data, offers,...","[1, 1, 5, 1, 1, 1, 1, 6, 1, 1, 1, 6, 1, 1, 1, ...",Latin
2,"[#VPN, #ummmaaa, #proud_be]","[1, 2, 1]",Latin
3,"[chandimal, .., uuu, thama, mulu, tem, ekama, ...","[5, 6, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2]",Latin
4,[sltgo],[5],Latin
...,...,...,...
13513,"[Me, mase, web, family, plus, packge, eke, tot...","[2, 2, 1, 1, 1, 1, 2, 1, 2, 5, 2, 2]",Latin
13514,"[munge, Dan, data, kapana, widiye, awulk, thiy...","[2, 2, 1, 2, 2, 2, 2]",Latin
13515,"[I, didn't, received, loyalty, data, past, mon...","[1, 1, 1, 1, 1, 1, 1, 6, 1, 1, 1, 1, 1, 1, 1, ...",Latin
13516,"[worst, service, ever, !, !, !, stolen, more, ...","[1, 1, 1, 6, 6, 6, 1, 1, 1, 5, 1, 1, 1, 1]",Latin


In [49]:
df_classified.loc[df_classified['class'] == 'Symbol']

,tokens,tags,class
178,"[1000, /=]","[5, 6]",Symbol
7988,"[1000, /=]","[5, 6]",Symbol


In [50]:
df_classified.loc[df_classified['class'] == 'Latin']

,tokens,tags,class
0,"[▁Amma, ge, ▁Ada, ray, ta, ❤, ️, E, ka, ▁Da, w...","[2, 2, 2, 2, 2, 6, 6, 2, 2, 2, 2, 2, 2, 2, 2, ...",Latin
1,"[We, need, IPL, Champions, leak, data, offers,...","[1, 1, 5, 1, 1, 1, 1, 6, 1, 1, 1, 6, 1, 1, 1, ...",Latin
2,"[#VPN, #ummmaaa, #proud_be]","[1, 2, 1]",Latin
3,"[chandimal, .., uuu, thama, mulu, tem, ekama, ...","[5, 6, 2, 2, 2, 1, 2, 2, 2, 2, 1, 2, 2]",Latin
4,[sltgo],[5],Latin
...,...,...,...
13513,"[Me, mase, web, family, plus, packge, eke, tot...","[2, 2, 1, 1, 1, 1, 2, 1, 2, 5, 2, 2]",Latin
13514,"[munge, Dan, data, kapana, widiye, awulk, thiy...","[2, 2, 1, 2, 2, 2, 2]",Latin
13515,"[I, didn't, received, loyalty, data, past, mon...","[1, 1, 1, 1, 1, 1, 1, 6, 1, 1, 1, 1, 1, 1, 1, ...",Latin
13516,"[worst, service, ever, !, !, !, stolen, more, ...","[1, 1, 1, 6, 6, 6, 1, 1, 1, 5, 1, 1, 1, 1]",Latin


In [51]:
df_classified.loc[df_classified['class'] == 'Sinhala']

,tokens,tags,class
16,"[▁හොඳ, ▁වැඩේ, ., ..., මෙ, ය, ැයි, ල, ගේ, ▁හැටි...","[0, 0, 6, 6, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, ...",Sinhala
19,"[හුම්, 😂]","[0, 6]",Sinhala
60,"[අංක, 3ක්, හරි, 😏]","[0, 0, 0, 6]",Sinhala
68,"[✋, ඉත්තෑව, අයින්, කරා]","[6, 0, 0, 0]",Sinhala
75,"[සුභ, පතනවා, මොබිටෙල්, වෙත]","[0, 0, 5, 0]",Sinhala
...,...,...,...
13474,"[අවුරුදු, 7ක්, පමන, බාවිතා, කරනවා, .]","[0, 0, 0, 0, 0, 6]",Sinhala
13477,"[අපි, පාවිච්චි, කරන, විදිහ, අපි, දන්නවා, .]","[0, 0, 0, 0, 0, 0, 6]",Sinhala
13478,"[ඉවර, වෙන්න, විදිහක්, නේ, .]","[0, 0, 0, 0, 6]",Sinhala
13479,"[මේක, මංකොල්ල, කැමක්]","[0, 0, 0]",Sinhala


In [52]:
df_classified.loc[df_classified['class'] == 'Mixed']

,tokens,tags,class
12,"[මොකක්ද, අප්ප, මේ, වෙනදට, වඩා, msg, එන්නෙ, mib...","[0, 0, 0, 0, 0, 1, 0, 5, 0, 0]",Mixed
13,"[Fiber, ඉල්ලලා, දැන්, අව්රුද්දක්, විතර, 😒, bll...","[1, 0, 0, 0, 0, 6, 2, 6]",Mixed
17,"[මොකුත්, එපා, ,, peak, /, off, peak, මඟුල, නැත...","[0, 0, 6, 1, 6, 1, 1, 0, 0, 0, 0, 0, 6]",Mixed
32,"[ඉගෙනුම්, ඉලක්ක, සපුරා, ගැනීමට, SLT, Online, l...","[0, 0, 0, 0, 5, 1, 1, 1, 0, 0, 6]",Mixed
33,"[www.slt.lk/esiphala, www.alkuppiya.lk, සුවිශේ...","[5, 5, 0, 1, 1, 0, 5, 5, 1, 1]",Mixed
...,...,...,...
13492,"[පුදුම, Customer, service, තමයි, ..]","[0, 1, 1, 0, 6]",Mixed
13493,"[Free, call, දෙපාරක්, දීලා, තියනවා, .]","[1, 1, 0, 0, 0, 6]",Mixed
13495,"[තුන්වෙනි, පාර, ගද්දි, සල්ලි, කැපෙනවා, .., වින...","[0, 0, 0, 0, 0, 6, 0, 0, 6, 5, 6, 1]",Mixed
13496,"[Dialog, ..., මොන, කෙහෙම්මලක්ද, ..., #678, #, ...","[5, 6, 0, 0, 6, 5, 6, 1, 0, 3, 0, 0, 1, 3, 0, ...",Mixed


In [53]:
df_classified['class'].value_counts()

Latin      9597
Mixed      1996
Sinhala    1923
Symbol        2
Name: class, dtype: int64

In [54]:
save_url = '/content/drive/Shareddrives/Lingua/Datasets/Sinhala-English Code Mixed Code Switched/sentence -script -latin-sinhala-mixed - sentence.csv'
df.to_csv(save_url, index=False)